<a href="https://colab.research.google.com/github/QuinnStraus/CSCI-1470-final/blob/main/DL_Final_Preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
pip install pypianoroll

     |████████████████████████████████| 5.6 MB 6.4 MB/s 
     |████████████████████████████████| 51 kB 6.2 MB/s 
  Created wheel for pretty-midi: filename=pretty_midi-0.2.9-py3-none-any.whl size=5591953 sha256=fdeb3c283f86fe34a26b1469bd07a6ffaf931fa9f96398aa46e053b93b17c3e0
  Stored in directory: /root/.cache/pip/wheels/ad/74/7c/a06473ca8dcb63efb98c1e67667ce39d52100f837835ea18fa
Successfully built pretty-midi


In [76]:
import pypianoroll
import os
import pandas as pd
import tensorflow as tf
import numpy as np
import json
import math
from music21 import *
import pdb


In [9]:
from google.colab import drive
drive.mount("/content/gdrive/")

Mounted at /content/gdrive/


In [10]:
!ls 'gdrive/Shared drives/CSCI1470 Final Project'
root_dir = 'gdrive/Shared drives/CSCI1470 Final Project'
data_dir = 'gdrive/Shared drives/CSCI1470 Final Project/lpd_5/lpd_5_cleansed'



 cleansed_ids.txt		   lpd_5	    msd_tagtraum_cd1.cls
'Final Write Up Reflection.gdoc'   midi_info.json   songs.txt


In [11]:
# Creates dict of song IDs with genres

genre_file_dir = os.path.join(root_dir, 'msd_tagtraum_cd1.cls')
ids = []
genres = []
with open(genre_file_dir) as f:
    line = f.readline()
    while line:
        if line[0] != '#':
            split = line.strip().split("\t")
            if len(split) == 2:
                ids.append(split[0])
                genres.append(split[1])
            elif len(split) == 3:
                ids.append(split[0]) 
                ids.append(split[0])
                genres.append(split[1])
                genres.append(split[2])
        line = f.readline()
genre_df = pd.DataFrame(data={"TrackID": ids, "Genre": genres})

genre_dict = genre_df.groupby('TrackID')['Genre'].apply(lambda x: x.tolist()).to_dict()

In [12]:
print(len(genre_dict))
print(genre_dict['TRAAAAK128F9318786'])

133676
['Pop_Rock']


In [13]:
# copied the path functions from the article
# Utility functions for retrieving paths
def msd_id_to_dirs(msd_id):
    """Given an MSD ID, generate the path prefix.
    E.g. TRABCD12345678 -> A/B/C/TRABCD12345678"""
    return os.path.join(msd_id[2], msd_id[3], msd_id[4], msd_id)

# Load the midi npz file from the LMD cleansed folder
def get_midi_npz_path(msd_id, midi_md5):
    return os.path.join(data_dir,
                        msd_id_to_dirs(msd_id), midi_md5 + '.npz')
    
# Load the midi file from the Music Dataset folder
#def get_midi_path(msd_id, midi_md5):
#    return os.path.join(music_dataset_lpd_dir,
#                        msd_id_to_dirs(msd_id), midi_md5 + '.mid')
    
# Open the cleansed ids - cleansed file ids : msd ids
cleansed_ids = pd.read_csv(os.path.join(root_dir, 'cleansed_ids.txt'), delimiter = '    ', header = None)
lpd_to_msd_ids = {a:b for a, b in zip(cleansed_ids[0], cleansed_ids[1])}
msd_to_lpd_ids = {a:b for a, b in zip(cleansed_ids[1], cleansed_ids[0])}
print(len(msd_to_lpd_ids))

# Creates list of all ids with Pop Rock genre tag

pop_ids = {}
for tag in msd_to_lpd_ids.keys():
  if tag in genre_dict:
    if 'Pop_Rock' in genre_dict.get(tag):
      pop_ids[tag] = msd_to_lpd_ids.get(tag)
print(len(pop_ids))


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:19: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.


21425
2423


In [ ]:
# DO NOT RUN!!

# converts npz files to midi files, no need to run again as all midi files already saved

for msd_file_name in cleansed_ids[1]:
  lpd_file_name = msd_to_lpd_ids[msd_file_name] 
  # msd_file_name ex: TRLOCTJ12903CDEC24, lpd_file_name: 6b16a2b1720d11263069a9a189785dfb
  npz_path = get_midi_npz_path(msd_file_name, lpd_file_name)
  multitrack = pypianoroll.load(npz_path)
  pm = pypianoroll.to_pretty_midi(multitrack)
  new_midi_path = npz_path[:-4] + '.mid'
  pypianoroll.write(new_midi_path, multitrack)

KeyboardInterrupt: ignored

In [ ]:
# Old preprocessing function, not used

# save songs to text file after tokenizing the midi files
allSongs = []
stopInd = 2
song_ind=0
# f = open(os.path.join(root_dir, "songs.txt"), "w")
for msd_file_name in cleansed_ids[1]:
  print(msd_file_name)
  elt_ind = 0
  lpd_file_name = msd_to_lpd_ids[msd_file_name]
  npz_path = get_midi_npz_path(msd_file_name, lpd_file_name)
  new_midi_path = npz_path[:-4] + '.mid'
  print("loaded data")
  midi = converter.parse(new_midi_path)
  #print("opened file")
  #open midi file and select piano part
  for part in instrument.partitionByInstrument(midi):
    if part.getInstrument().instrumentName == 'Piano':
      curSong = []
      #add notes, chords, and rests to list of strings
      for i in range(min(len(part), WINDOW_SZ)):
        elt = part[i]
        if isinstance(elt, note.Note):
          toWrite = str(elt.pitch) + 'd' + str(elt.duration)[27:-1] + ' br '
          curSong.append(toWrite)
          # f.write(toWrite)
        elif isinstance(elt, chord.Chord):
          dur = str(elt.duration)[27:-1]
          if dur == '0.0' and isinstance(part[elt_ind+1], note.Rest):
              dur = str(part[elt_ind+1].duration)[27:-1]
          toWrite = ''
          for nt in elt.pitches:
            toWrite += str(nt) + 'd' + dur + ' '
          toWrite += ' br '
          # f.write(toWrite)
        elt_ind += 1
  song_ind+=1
  # f.write('\n')
  if song_ind % stopInd == 0:
    print(song_ind)
    # f.close()
    break

In [74]:
# These hyperparameters determine what section of the song is encoded

START_IND = 0
NUM_TOKENS = 210

# Converts file name to pypianoroll
def get_midi(msd):
  lpd_file_name = pop_ids[msd]
  npz_path = get_midi_npz_path(msd, lpd_file_name)
  new_midi_path = npz_path[:-4] + '.npz'
  return(pypianoroll.load(new_midi_path))

# Helper function that combines instruments, returns matrix of pitches
def merge_instruments(multitrack):
  size = multitrack.get_max_length()
  out = np.zeros([size, 128])
  for track in multitrack.pad_to_same().tracks:
    if track.is_drum == False:
      out += track
  return out

# Helper function that takes an array of notes and converts into string of tokens of pitch and duration
def tokenize(part, spaces = False): 
  if spaces:
    space = ' '
  else:
    space = ''
  out = ''
  br_count = 0
  tokens = 0
  note = START_IND
  # Counts out and produces tokens
  while tokens <= NUM_TOKENS and note < len(part):
    out_note = ''
    pitches = part[note].nonzero()[0]
    for pitch in pitches:
      if note == START_IND or part[note - 1, pitch] == 0:
        duration = 1
        holding = True
        # Find duration of note
        while holding:
          if note + duration < len(part):
            if part[note + duration, pitch] != 0:
              duration += 1
            else:
              holding = False
          else:
            holding = False
        out_note += str(pitch) + space + 'd' + str(duration) + ' '
        if spaces:
          tokens += 2
        else:
          tokens += 1
    # Add breaks
    if out_note != '':
      if br_count != 0:
        out_note = 'br' + space + 'd' + str(br_count) + ' ' + out_note
        br_count = 0
        if spaces:
          tokens += 2
        else:
          tokens += 1
    br_count += 1
    out += out_note
    note += 1
  return out

In [79]:
# pypianoroll converting files to tokens

# save songs to text file after tokenizing the midi files
allSongs = []
stopInd = 50
song_ind = 0
f = open(os.path.join(root_dir, "songs.txt"), "w")
for msd in pop_ids.keys():
  elt_ind = 0
  midi = get_midi(msd)
  if song_ind %10 == 0: 
    print(song_ind)
  #open midi file and select piano part
  res = 8
  rolls = midi.set_resolution(res)
  part = merge_instruments(rolls)
  #pypianoroll.plot(pypianoroll.StandardTrack(pianoroll = part[10*4*res:15*4*res]))
  output = tokenize(part, spaces = False)
  if output != '':
    f.write(output)
    f.write('\n') 
  song_ind+=1
  if song_ind % stopInd == 0:
    print(song_ind)
    f.close()
    break

0
10
20
30
40
50


In [58]:
# Takes in a string of tokens representing one song, transforms into a pianoroll matrix
def token_to_music(tokens):
  token_list = tokens.split()
  mat = np.full([1, 128], False)
  note = 0
  for tk in token_list:
    pitch = tk.split('d')[0]
    dur = int(tk.split('d')[1])
    if pitch == 'br':
      note += dur
    else:
      for i in range(dur):
        while note + i >= mat.shape[0]:
          mat = np.append(mat, np.full([1, 128], False), axis = 0)
        mat[note + i, int(pitch)] = True
  return mat

In [ ]:
# Tests token_to_music
song_ind = 0
for msd in pop_ids.keys():
  print(msd)
  roll = get_midi('TRUZHFD128F429C33B')
  roll = roll.set_resolution(4)
  part = merge_instruments(roll)[0:80]
  tokenized = tokenize(part)
  other_roll = pypianoroll.binarize(pypianoroll.Track(pianoroll = part)).pianoroll
  #print(token_to_music(tokenized).shape)
  #print(other_roll)
  print(np.all(token_to_music(tokenized) == other_roll))
  song_ind += 1
  if song_ind % 10 == 0:
    break

In [ ]:
track = pypianoroll.BinaryTrack(pianoroll=token_to_music("") #fill in "" with tokens 
pypianoroll.write(os.path.join(root_dir, "output.mid"), track))